![img](https://github.com/facebookresearch/fairseq/raw/nllb/nllb.png?raw=true)

# FairSeq and FairScale Installation
https://github.com/facebookresearch/fairseq/blob/nllb/INSTALL.md

In [1]:
# ! pip install fairscale
# ! pip install sentencepiece
# ! git clone https://github.com/facebookresearch/fairseq
# %cd fairseq
# ! git checkout nllb -f
import sys
sys.path.insert(1, "/Users/junehyeok/HYEOKIKI/SOMA/chatbot/fairseq/fairseq")
sys.path.insert(1, "/Users/junehyeok/HYEOKIKI/SOMA/chatbot/fairseq")
sys.path.insert(1, ".")
sys.path

# %cd /Users/junehyeok/HYEOKIKI/SOMA/chatbot/fairseq
# !pip install -e .
# %cd -

['/Users/junehyeok/HYEOKIKI/SOMA/chatbot',
 '.',
 '/Users/junehyeok/HYEOKIKI/SOMA/chatbot/fairseq',
 '/Users/junehyeok/HYEOKIKI/SOMA/chatbot/fairseq/fairseq',
 '/Users/junehyeok/miniforge3/envs/translate/lib/python38.zip',
 '/Users/junehyeok/miniforge3/envs/translate/lib/python3.8',
 '/Users/junehyeok/miniforge3/envs/translate/lib/python3.8/lib-dynload',
 '',
 '/Users/junehyeok/miniforge3/envs/translate/lib/python3.8/site-packages',
 '/Users/junehyeok/HYEOKIKI/SOMA/chatbot/fairseq']

# Download the Model Checkpoint

In [ ]:
%cd /Users/junehyeok/HYEOKIKI/SOMA/chatbot
! wget --trust-server-names https://tinyurl.com/nllb200densedst600mcheckpoint


/content/drive/Othercomputers/MacBookPro/chatbot
--2022-07-27 18:18:21--  https://tinyurl.com/nllb200densedst600mcheckpoint
Resolving tinyurl.com (tinyurl.com)... 104.20.138.65, 172.67.1.225, 104.20.139.65, ...
Connecting to tinyurl.com (tinyurl.com)|104.20.138.65|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dl.fbaipublicfiles.com/large_objects/nllb/models/nllb_200_dense_distill_600m/checkpoint.pt [following]
--2022-07-27 18:18:21--  https://dl.fbaipublicfiles.com/large_objects/nllb/models/nllb_200_dense_distill_600m/checkpoint.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4740531293 (4.4G) [binary/octet-stream]
Saving to: ‘checkpoint.pt.1’

checkpoint.pt.1     100%[===================>]   4.41G  32.3MB/s    in 2m 25s  

2

# SentencePiece Command Line Tools installation

In [10]:
# !apt-get -y install sentencepiece
# !pip install sentencepiece
# ! git clone https://github.com/google/sentencepiece.git
# ! git clone https://github.com/pluiez/NLLB-inference

# Pick your Language Codes  
https://github.com/facebookresearch/flores/tree/main/flores200#languages-in-flores-200

In [2]:
%cd /Users/junehyeok/HYEOKIKI/SOMA/chatbot/NLLB-inference
# !conda install -y pyarrow

import ast
import fileinput
import logging
import math
import os
import sys
import time
from argparse import Namespace
from collections import namedtuple

import numpy as np
import torch

from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from fairseq.dataclass.configs import FairseqConfig
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from fairseq.token_generation_constraints import pack_constraints, unpack_constraints
from fairseq_cli.generate import get_symbols_to_strip_from_output
from fairseq_cli.interactive import *

import sentencepiece as spm


def gen_cfg(item):
  if not isinstance(item, dict):
    return item
 
  class cfg(object) : pass
  for k, v in item.items():
    setattr(cfg, k, gen_cfg(v))
  return cfg

def encode_fn(x):
    if tokenizer is not None:
        x = tokenizer.encode(x)
    if bpe is not None:
        x = bpe.encode(x)
    return x

def decode_fn(x):
    if bpe is not None:
        x = bpe.decode(x)
    if tokenizer is not None:
        x = tokenizer.decode(x)
    return x

def translate(cfg) : 
  # logger.info("Type the input sentence and press return:")
  start_id = 0
  # for inputs in buffered_read(cfg.interactive.input, cfg.interactive.buffer_size):
  inputs = [" ".join(sp.EncodeAsPieces(x)) for x in cfg.interactive.input]
  results = []
  for batch in make_batches(inputs, cfg, task, max_positions, encode_fn):
      bsz = batch.src_tokens.size(0)
      src_tokens = batch.src_tokens
      src_lengths = batch.src_lengths
      constraints = batch.constraints
      if use_cuda:
          src_tokens = src_tokens.cuda()
          src_lengths = src_lengths.cuda()
          if constraints is not None:
              constraints = constraints.cuda()

      sample = {
          "net_input": {
              "src_tokens": src_tokens,
              "src_lengths": src_lengths,
          },
      }
      translate_start_time = time.time()
      translations = task.inference_step(
          generator, models, sample, constraints=constraints
      )
      translate_time = time.time() - translate_start_time
      # total_translate_time += translate_time
      list_constraints = [[] for _ in range(bsz)]
      if cfg.generation.constraints:
          list_constraints = [unpack_constraints(c) for c in constraints]
      for i, (id, hypos) in enumerate(zip(batch.ids.tolist(), translations)):
          src_tokens_i = utils.strip_pad(src_tokens[i], tgt_dict.pad())
          constraints = list_constraints[i]
          results.append(
              (
                  start_id + id,
                  src_tokens_i,
                  hypos,
                  {
                      "constraints": constraints,
                      "time": translate_time / len(translations),
                  },
              )
          )

      translated = []
      # sort output to match input order
      for id_, src_tokens, hypos, info in sorted(results, key=lambda x: x[0]):
          src_str = ""
          if src_dict is not None:
              src_str = src_dict.string(src_tokens, cfg.common_eval.post_process)
              
          translated += [ sp.DecodePieces(utils.post_process_prediction(
                  hypo_tokens=hypo["tokens"].int().cpu(),
                  src_str=src_str,
                  alignment=hypo["alignment"],
                  align_dict=align_dict,
                  tgt_dict=tgt_dict,
                  remove_bpe=cfg.common_eval.post_process,
                  extra_symbols_to_ignore=get_symbols_to_strip_from_output(generator),
              )[1].split(' ')) for hypo in hypos[: min(len(hypos), cfg.generation.nbest)]]
          # Process top predictions
          

      # update running id_ counter
      start_id += len(inputs)



  return translated

/Users/junehyeok/HYEOKIKI/SOMA/chatbot/NLLB-inference


/Users/junehyeok/miniforge3/envs/translate/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-16 13:19:29 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [3]:
cfg_dict = {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma', 'log_nvidia_smi': False, 'use_tutel_moe': False}, 'common_eval': {'_name': None, 'path': 'checkpoint.pt', 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None, 'is_moe': False, 'moe_generation': False}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_num_procs': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_port': -1, 'device_id': 0, 'distributed_no_spawn': False, 'ddp_backend': 'pytorch_ddp', 'ddp_comm_hook': 'none', 'bucket_cap_mb': 25, 'fix_batches_to_gpus': False, 'find_unused_parameters': False, 'gradient_as_bucket_view': False, 'fast_stat_sync': False, 'heartbeat_timeout': -1, 'broadcast_buffers': False, 'slowmo_momentum': None, 'slowmo_base_algorithm': 'localsgd', 'localsgd_frequency': 3, 'nprocs_per_node': 1, 'pipeline_model_parallel': False, 'pipeline_balance': None, 'pipeline_devices': None, 'pipeline_chunks': 0, 'pipeline_encoder_balance': None, 'pipeline_encoder_devices': None, 'pipeline_decoder_balance': None, 'pipeline_decoder_devices': None, 'pipeline_checkpoint': 'never', 'zero_sharding': 'none', 'fp16': False, 'memory_efficient_fp16': False, 'tpu': False, 'no_reshard_after_forward': False, 'fp32_reduce_scatter': False, 'cpu_offload': False, 'use_sharded_state': False, 'not_fsdp_flatten_parameters': False, 'freeze_up_to_layer': None}, 'dataset': {'_name': None, 'num_workers': 1, 'num_workers_valid': 0, 'skip_invalid_size_inputs_valid_test': False, 'max_tokens': 8192, 'batch_size': 1024, 'required_batch_size_multiple': 8, 'required_seq_len_multiple': 1, 'dataset_impl': None, 'data_buffer_size': 10, 'train_subset': 'train', 'valid_subset': 'valid', 'combine_valid_subsets': None, 'ignore_unused_valid_subsets': False, 'validate_interval': 1, 'validate_interval_updates': 0, 'validate_after_updates': 0, 'fixed_validation_seed': None, 'disable_validation': False, 'max_tokens_valid': 8192, 'batch_size_valid': 1024, 'max_valid_steps': None, 'curriculum': 0, 'gen_subset': 'test', 'num_shards': 1, 'shard_id': 0, 'grouped_shuffling': False, 'update_epoch_batch_itr': False, 'update_ordered_indices_seed': False}, 'optimization': {'_name': None, 'max_epoch': 0, 'max_update': 0, 'stop_time_hours': 0.0, 'clip_norm': 0.0, 'sentence_avg': False, 'update_freq': [1], 'lr': [0.25], 'stop_min_lr': -1.0, 'use_bmuf': False, 'train_with_epoch_remainder_batch': False}, 'checkpoint': {'_name': None, 'save_dir': 'checkpoints', 'restore_file': 'checkpoint_last.pt', 'continue_once': None, 'finetune_from_model': None, 'ignore_suffix': False, 'reset_dataloader': False, 'reset_lr_scheduler': False, 'reset_meters': False, 'reset_optimizer': False, 'optimizer_overrides': '{}', 'save_interval': 1, 'save_interval_updates': 0, 'keep_interval_updates': -1, 'keep_interval_updates_pattern': -1, 'keep_last_epochs': -1, 'keep_best_checkpoints': -1, 'no_save': False, 'no_epoch_checkpoints': False, 'no_last_checkpoints': False, 'no_best_checkpoints': False, 'no_save_optimizer_state': False, 'no_save_optimizer_state_on_training_finished': False, 'synchronize_checkpoints_before_copy': False, 'symlink_best_and_last_checkpoints': False, 'best_checkpoint_metric': 'loss', 'maximize_best_checkpoint_metric': False, 'patience': -1, 'checkpoint_suffix': '', 'checkpoint_shard_count': 1, 'load_checkpoint_on_all_dp_ranks': False, 'write_checkpoints_asynchronously': False, 's3_upload_path': None, 'replication_count': 1, 'model_parallel_size': 1}, 'bmuf': {'_name': None, 'block_lr': 1.0, 'block_momentum': 0.875, 'global_sync_iter': 50, 'warmup_iterations': 500, 'use_nbm': False, 'average_sync': False, 'distributed_world_size': 1}, 'generation': {'_name': None, 'beam': 4, 'nbest': 1, 'max_len_a': 0.0, 'max_len_b': 200, 'min_len': 1, 'match_source_len': False, 'unnormalized': False, 'no_early_stop': False, 'no_beamable_mm': False, 'lenpen': 1.0, 'unkpen': 0.0, 'replace_unk': None, 'sacrebleu': False, 'score_reference': False, 'prefix_size': 0, 'no_repeat_ngram_size': 0, 'sampling': False, 'sampling_topk': -1, 'sampling_topp': -1.0, 'constraints': None, 'temperature': 1.0, 'diverse_beam_groups': -1, 'diverse_beam_strength': 0.5, 'diversity_rate': -1.0, 'print_alignment': None, 'print_step': False, 'lm_path': None, 'lm_weight': 0.0, 'iter_decode_eos_penalty': 0.0, 'iter_decode_max_iter': 10, 'iter_decode_force_max_iter': False, 'iter_decode_with_beam': 1, 'iter_decode_with_external_reranker': False, 'retain_iter_history': False, 'retain_dropout': False, 'retain_dropout_modules': None, 'decoding_format': None, 'no_seed_provided': False}, 'eval_lm': {'_name': None, 'output_word_probs': False, 'output_word_stats': False, 'context_window': 0, 'softmax_batch': 9223372036854775807, 'stats_path': None, 'max_valid_steps': None}, 'interactive': {'_name': None, 'buffer_size': 100000, 'input': '-'}, 'model': None, 'task': Namespace(_name='translation_multi_simple_epoch', add_data_source_prefix_tags=True, add_ssl_task_tokens=False, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, batch_size=1024, batch_size_valid=1024, beam=4, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, buffer_size=100000, checkpoint_shard_count=1, checkpoint_suffix='', combine_valid_subsets=None, constraints=None, continue_once=None, cpu=False, cpu_offload=False, criterion='cross_entropy', curriculum=0, data='.', data_buffer_size=10, dataset_impl=None, ddp_backend='pytorch_ddp', ddp_comm_hook='none', decoder_langtok=True, decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, distributed_world_size=1, diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, enable_lang_ids=False, enable_m2m_validation=False, enable_reservsed_directions_shared_datasets=False, encoder_langtok='src', eos=2, eval_lang_pairs=None, extra_data=None, extra_lang_pairs=None, fast_stat_sync=False, find_unused_parameters=False, finetune_dict_specs=None, finetune_from_model=None, fix_batches_to_gpus=False, fixed_dictionary='./dictionary.txt', fixed_validation_seed=None, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, fp32_reduce_scatter=False, freeze_up_to_layer=None, gen_subset='test', gradient_as_bucket_view=False, grouped_shuffling=False, heartbeat_timeout=-1, ignore_mmt_main_data=False, ignore_suffix=False, ignore_unused_valid_subsets=False, input='-', insert=0.0, is_moe=False, iter_decode_eos_penalty=0.0, iter_decode_force_max_iter=False, iter_decode_max_iter=10, iter_decode_with_beam=1, iter_decode_with_external_reranker=False, keep_best_checkpoints=-1, keep_inference_langtok=False, keep_interval_updates=-1, keep_interval_updates_pattern=-1, keep_last_epochs=-1, lang_dict=None, lang_pairs='eng_Latn-kor_Hang', lang_tok_replacing_bos_eos=False, lang_tok_style='multilingual', langs=['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'pes_Arab', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'knc_Arab', 'knc_Latn', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kon_Latn', 'kor_Hang', 'kmr_Latn', 'lao_Laoo', 'lvs_Latn', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Latn', 'mkd_Cyrl', 'plt_Latn', 'mlt_Latn', 'mni_Beng', 'khk_Cyrl', 'mos_Latn', 'mri_Latn', 'zsm_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'gaz_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pol_Latn', 'por_Latn', 'prs_Arab', 'pbt_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Beng', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'als_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'taq_Latn', 'taq_Tfng', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zul_Latn'], langtoks=None, langtoks_specs=['main'], left_pad_source='True', left_pad_target='False', lenpen=1.0, lm_path=None, lm_weight=0.0, load_alignments=False, load_checkpoint_on_all_dp_ranks=False, localsgd_frequency=3, log_file=None, log_format=None, log_interval=100, log_nvidia_smi=False, lr_scheduler='fixed', lr_shrink=0.1, mask=0.1, mask_length='subword', mask_random=0.0, match_source_len=False, max_len_a=0, max_len_b=200, max_source_positions=1024, max_target_positions=1024, max_tokens=8192, max_tokens_valid=8192, max_valid_steps=None, maximize_best_checkpoint_metric=False, memory_efficient_bf16=False, memory_efficient_fp16=False, min_len=1, min_loss_scale=0.0001, mixed_multitask_denoising_prob=0.5, model_overrides='{}', model_parallel_size=1, moe_generation=False, nbest=1, no_beamable_mm=False, no_best_checkpoints=False, no_early_stop=False, no_epoch_checkpoints=False, no_last_checkpoints=False, no_progress_bar=False, no_repeat_ngram_size=0, no_reshard_after_forward=False, no_save=False, no_save_optimizer_state=False, no_save_optimizer_state_on_training_finished=False, no_seed_provided=False, not_fsdp_flatten_parameters=False, nprocs_per_node=1, num_shards=1, num_workers=1, num_workers_valid=0, on_cpu_convert_precision=False, one_dataset_per_batch=False, optimizer=None, optimizer_overrides='{}', pad=1, pad_to_fixed_length=False, path='checkpoint.pt', patience=-1, permute=0.0, permute_sentences=0.0, pipeline_balance=None, pipeline_checkpoint='never', pipeline_chunks=0, pipeline_decoder_balance=None, pipeline_decoder_devices=None, pipeline_devices=None, pipeline_encoder_balance=None, pipeline_encoder_devices=None, pipeline_model_parallel=False, plasma_path='/tmp/plasma', poisson_lambda=3.0, post_process=None, prefix_size=0, print_alignment=None, print_step=False, profile=False, quantization_config_path=None, quiet=False, replace_length=1, replace_unk=None, replication_count=1, required_batch_size_multiple=8, required_seq_len_multiple=1, reset_dataloader=False, reset_logging=False, reset_lr_scheduler=False, reset_meters=False, reset_optimizer=False, restore_file='checkpoint_last.pt', results_path=None, retain_dropout=False, retain_dropout_modules=None, retain_iter_history=False, rotate=0.0, s3_upload_path=None, sacrebleu=False, sample_break_mode='eos', sampling=False, sampling_method='concat', sampling_temperature=1.5, sampling_topk=-1, sampling_topp=-1.0, sampling_weights=None, sampling_weights_from_file=None, save_dir='checkpoints', save_interval=1, save_interval_updates=0, score_reference=False, scoring='bleu', seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, slowmo_base_algorithm='localsgd', slowmo_momentum=None, source_dict=None, source_lang='eng_Latn', suppress_crashes=False, symlink_best_and_last_checkpoints=False, synchronize_checkpoints_before_copy=False, target_dict=None, target_lang='kor_Hang', task='translation_multi_simple_epoch', temperature=1.0, tensorboard_logdir=None, threshold_loss_scale=None, tokenizer=None, tokens_per_sample=512, tpu=False, train_subset='train', truncate_source=False, unk=3, unkpen=0, unnormalized=False, update_epoch_batch_itr=False, update_ordered_indices_seed=False, upsample_primary=1, use_local_shard_size=False, use_plasma_view=False, use_sharded_state=False, use_tutel_moe=False, user_dir=None, valid_subset='valid', validate_after_updates=0, validate_interval=1, validate_interval_updates=0, virtual_data_size=None, virtual_epoch_size=None, wandb_project=None, warmup_updates=0, write_checkpoints_asynchronously=False, zero_sharding='none'), 'criterion': {'_name': 'cross_entropy', 'sentence_avg': True}, 'optimizer': None, 'lr_scheduler': {'_name': 'fixed', 'force_anneal': None, 'lr_shrink': 0.1, 'warmup_updates': 0, 'lr': [0.25]}, 'scoring': {'_name': 'bleu', 'pad': 1, 'eos': 2, 'unk': 3}, 'bpe': None, 'tokenizer': None, 'ema': {'_name': None, 'store_ema': False, 'ema_decay': 0.9999, 'ema_start_update': 0, 'ema_seed_model': None, 'ema_update_freq': 1, 'ema_fp32': False}}

print(cfg_dict['checkpoint'])

cfg = gen_cfg(cfg_dict)
# setattr(a, 'b', 'value')
# a.b

if isinstance(cfg, Namespace):
    cfg = convert_namespace_to_omegaconf(cfg)

start_time = time.time()
total_translate_time = 0

utils.import_user_module(cfg.common)

if cfg.interactive.buffer_size < 1:
    cfg.interactive.buffer_size = 1
if cfg.dataset.max_tokens is None and cfg.dataset.batch_size is None:
    cfg.dataset.batch_size = 1

assert (
    not cfg.generation.sampling or cfg.generation.nbest == cfg.generation.beam
), "--sampling requires --nbest to be equal to --beam"
assert (
    not cfg.dataset.batch_size
    or cfg.dataset.batch_size <= cfg.interactive.buffer_size
), "--batch-size cannot be larger than --buffer-size"

# logger.info(cfg)

# Fix seed for stochastic decoding
if cfg.common.seed is not None and not cfg.generation.no_seed_provided:
    np.random.seed(cfg.common.seed)
    utils.set_torch_seed(cfg.common.seed)

use_cuda = torch.cuda.is_available() and not cfg.common.cpu

# Setup task, e.g., translation
task = tasks.setup_task(cfg.task)

# Load ensemble
overrides = ast.literal_eval(cfg.common_eval.model_overrides)
# logger.info("loading model(s) from {}".format(cfg.common_eval.path))
models, _model_args = checkpoint_utils.load_model_ensemble(
    utils.split_paths(cfg.common_eval.path),
    arg_overrides=overrides,
    task=task,
    suffix=cfg.checkpoint.checkpoint_suffix,
    strict=(cfg.checkpoint.checkpoint_shard_count == 1),
    num_shards=cfg.checkpoint.checkpoint_shard_count,
)

# Set dictionaries
src_dict = task.source_dictionary
tgt_dict = task.target_dictionary

# Optimize ensemble for generation
for model in models:
    if model is None:
        continue
    if cfg.common.fp16:
        model.half()
    if use_cuda and not cfg.distributed_training.pipeline_model_parallel:
        model.cuda()
    model.prepare_for_inference_(cfg)

# Initialize generator
generator = task.build_generator(models, cfg.generation)

# Handle tokenization and BPE
tokenizer = task.build_tokenizer(cfg.tokenizer)
bpe = task.build_bpe(cfg.bpe)

# Load alignment dictionary for unknown word replacement
# (None if no unknown word replacement, empty if no path to align dictionary)
align_dict = utils.load_align_dict(cfg.generation.replace_unk)

max_positions = utils.resolve_max_positions(
    task.max_positions(), *[model.max_positions() for model in models]
)

if cfg.generation.constraints:
    logger.warning(
        "NOTE: Constrained decoding currently assumes a shared subword vocabulary."
    )

if cfg.interactive.buffer_size > 1:
    logger.info("Sentence buffer size: %s", cfg.interactive.buffer_size)
logger.info("NOTE: hypothesis and token scores are output in base 2")

sp = spm.SentencePieceProcessor()

sp.load('preprocess/flores200_sacrebleu_tokenizer_spm.model')


2022-08-16 13:19:42 | INFO | fairseq.data.multilingual.multilingual_data_manager | parsed the language list as they are ordered in the option: ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'pes_Arab', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye

{'_name': None, 'save_dir': 'checkpoints', 'restore_file': 'checkpoint_last.pt', 'continue_once': None, 'finetune_from_model': None, 'ignore_suffix': False, 'reset_dataloader': False, 'reset_lr_scheduler': False, 'reset_meters': False, 'reset_optimizer': False, 'optimizer_overrides': '{}', 'save_interval': 1, 'save_interval_updates': 0, 'keep_interval_updates': -1, 'keep_interval_updates_pattern': -1, 'keep_last_epochs': -1, 'keep_best_checkpoints': -1, 'no_save': False, 'no_epoch_checkpoints': False, 'no_last_checkpoints': False, 'no_best_checkpoints': False, 'no_save_optimizer_state': False, 'no_save_optimizer_state_on_training_finished': False, 'synchronize_checkpoints_before_copy': False, 'symlink_best_and_last_checkpoints': False, 'best_checkpoint_metric': 'loss', 'maximize_best_checkpoint_metric': False, 'patience': -1, 'checkpoint_suffix': '', 'checkpoint_shard_count': 1, 'load_checkpoint_on_all_dp_ranks': False, 'write_checkpoints_asynchronously': False, 's3_upload_path': None,

2022-08-16 13:19:42 | INFO | fairseq.checkpoint_utils | load_model_ensemble_and_task is_moe=False
2022-08-16 13:19:44 | INFO | fairseq.checkpoint_utils | Rank 0: Done reading from disk
2022-08-16 13:19:51 | INFO | fairseq.checkpoint_utils | Done loading state dict
2022-08-16 13:19:51 | INFO | fairseq_cli.interactive | Sentence buffer size: 100000
2022-08-16 13:19:51 | INFO | fairseq_cli.interactive | NOTE: hypothesis and token scores are output in base 2


True

<h2> example

In [4]:
txt = """Hello, Friendship Restaurant. May I help you?
I am going to celebrate my birthday with you all in a night club this year, What do you say?
What do you do for a living?
What a nice garden! Look at these tulips! Did you watch television last night, Charles? Hey, Vernassa.
Why be angry look? I'm forming a music band."""

cfg.interactive.input = txt.split('\n')

result = translate(cfg)

result

2022-08-16 13:19:51 | INFO | fairseq.tasks.translation_multi_simple_epoch | start batch sampler: mem usage: used=4367.109375Mb; avail=8965.421875Mb
2022-08-16 13:19:51 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler order indices time: 0:00:00.001892
2022-08-16 13:19:51 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=4367.796875Mb; avail=8964.859375Mb
2022-08-16 13:19:51 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler batch_by_size time: 0:00:00.033484
2022-08-16 13:19:51 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] per epoch batch_sampler set-up time: 0:00:00.035898
2022-08-16 13:19:51 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=4416.09375Mb; avail=8917.84375Mb
/Users/junehyeok/miniforge3/envs/translate/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently 

['안녕하세요, 프렌디십 레스토랑, 도와드릴까요?',
 '올해는 나랑 함께 나이트클럽에서 생일 축하할 거야?',
 '무슨 일을 하는 거야?',
 '예쁜 정원이야, 툴루푸스 좀 봐요.',
 '왜 화를 내는 거야?']

<h1>pickle read

In [37]:
from dataclasses import dataclass
import pickle
import pandas as pd

data = []
with open("/Users/junehyeok/HYEOKIKI/SOMA/chatbot/dataset/train_dialogues.pickle", 'rb') as f:
    data = pickle.load(f)


print(len(data))
print(len(data[1]))

52610
6


In [38]:
feather_path = '/Users/junehyeok/HYEOKIKI/SOMA/chatbot/NLLB-inference/transed.feather'
f = pd.read_feather(feather_path)
print(len(f))
s = f.iloc[0]
s.apply(lambda x: eval(x))
print(s[0])

df.to_feather('transed.feather')

51623
['짐, 저녁 식사 후 몇 개의 맥주를 마시면 어떨까?', '그건 유혹적이지만 우리 몸에 좋지 않습니다.', '무슨 뜻이야? 그것은 우리를 편안하게 할 것입니다.', '정말 그렇게 생각하시나요? 난 그렇지 않아요. 그건 우리를 비만시키고 멍청하게 행동하게 만들죠.', '네 말이 맞다고 생각하지만 우리가 어떻게 해야 할까요?', '헬스장까지 걸어서 가창을 연주하고 친구들을 만날 수 있다고 제안합니다.', '좋은 아이디어요. 메리와 살리가 자주 핑핑핑을 하기 위해 그곳에 가는 거 들었어요. 어쩌면 우리는 그들과 함께 4번을 할 수 있을지도 모릅니다.', '좋은 소리! 그들이 원한다면, 우리는 그들과 함께 춤을 출 수 있습니다. 그것은 훌륭한 운동과 재미, 또한.', '좋아, 이제 가자.', '- 알았어']


In [39]:
import pandas as pd

file_name = 'transed.feather'
df = pd.DataFrame(columns=['data'])
if os.path.exists(file_name):
    df = pd.read_feather(file_name)

# df.index.name = 'id'
for d in data[len(df):]:
    cfg.interactive.input = d
    result = translate(cfg)
    df.loc[len(df)] = str(result)
    print(len(df))
df.to_feather('transed.feather')

#     break

# df

2022-08-18 18:02:02 | INFO | fairseq.tasks.translation_multi_simple_epoch | start batch sampler: mem usage: used=7279.6875Mb; avail=4808.109375Mb
2022-08-18 18:02:02 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler order indices time: 0:00:00.000702
2022-08-18 18:02:02 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=7279.71875Mb; avail=4808.15625Mb
2022-08-18 18:02:02 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler batch_by_size time: 0:00:00.000094
2022-08-18 18:02:02 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] per epoch batch_sampler set-up time: 0:00:00.001419
2022-08-18 18:02:02 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=7279.71875Mb; avail=4808.15625Mb
2022-08-18 18:02:09 | INFO | fairseq.tasks.translation_multi_simple_epoch | start batch sampler: mem usage: used=8015.984375Mb; avail=5590.828125Mb
2022-08-18 18:02:09 | INFO | fairseq.tasks.translation_

51624


2022-08-18 18:02:14 | INFO | fairseq.tasks.translation_multi_simple_epoch | start batch sampler: mem usage: used=8025.765625Mb; avail=5574.125Mb
2022-08-18 18:02:14 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler order indices time: 0:00:00.000472
2022-08-18 18:02:14 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=8025.765625Mb; avail=5574.125Mb
2022-08-18 18:02:14 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler batch_by_size time: 0:00:00.000057
2022-08-18 18:02:14 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] per epoch batch_sampler set-up time: 0:00:00.001119
2022-08-18 18:02:14 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=8025.765625Mb; avail=5574.125Mb


51625


2022-08-18 18:02:20 | INFO | fairseq.tasks.translation_multi_simple_epoch | start batch sampler: mem usage: used=8013.296875Mb; avail=5590.25Mb
2022-08-18 18:02:20 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler order indices time: 0:00:00.000457
2022-08-18 18:02:20 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=8013.4375Mb; avail=5590.546875Mb
2022-08-18 18:02:20 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler batch_by_size time: 0:00:00.000055
2022-08-18 18:02:20 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] per epoch batch_sampler set-up time: 0:00:00.001123
2022-08-18 18:02:20 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=8013.5Mb; avail=5590.65625Mb


...

52607


2022-08-18 19:47:50 | INFO | fairseq.tasks.translation_multi_simple_epoch | start batch sampler: mem usage: used=7300.75Mb; avail=4901.53125Mb
2022-08-18 19:47:50 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler order indices time: 0:00:00.000447
2022-08-18 19:47:50 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=7300.75Mb; avail=4901.53125Mb
2022-08-18 19:47:50 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler batch_by_size time: 0:00:00.000053
2022-08-18 19:47:50 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] per epoch batch_sampler set-up time: 0:00:00.001045
2022-08-18 19:47:50 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=7300.75Mb; avail=4901.53125Mb


52608


2022-08-18 19:47:59 | INFO | fairseq.tasks.translation_multi_simple_epoch | start batch sampler: mem usage: used=7183.703125Mb; avail=4997.0625Mb
2022-08-18 19:47:59 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler order indices time: 0:00:00.000455
2022-08-18 19:47:59 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=7183.703125Mb; avail=4997.0625Mb
2022-08-18 19:47:59 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] @batch_sampler batch_by_size time: 0:00:00.000054
2022-08-18 19:47:59 | INFO | fairseq.tasks.translation_multi_simple_epoch | [None] per epoch batch_sampler set-up time: 0:00:00.001097
2022-08-18 19:47:59 | INFO | fairseq.tasks.translation_multi_simple_epoch | mem usage: used=7183.703125Mb; avail=4997.0625Mb


52609
52610


In [ ]:
df2 = pd.read_feather('transed.feather')
df2['data'].apply(lambda x: eval(x))

custom train pickle생성


In [41]:
import pandas as pd

file_name = 'transed.feather'
df = pd.read_feather(file_name)
print(len(df))

52610


In [48]:
df.iloc[4].to_list()

["['존, 멋진 스케이트, 새가?', '그래, 방금 가져왔어요. 저는 커뮤니티 리그에서 아이스하키를 시작했어요. 그래서 마침내 새로운 스케이트를 샀어요.', '어떤 포지션을 맡고 있나요?', '난 수비수야, 너무 재미있어요. 수비에서 그렇게 빨리 스케이트를 타지 않아도 되죠.', '네, 넌 꽤 큰 놈이야 내가 골키퍼로 뛰는 거야', '어떤 팀인가요?', '로켓스', '내주 너희들랑 놀러 갈 것 같아 연습하러 가야 돼', '좋아, 나중에 보자.']"]